In [ ]:
!pip install jinja2

In [ ]:
import pathlib

import jinja2
from jinja2 import Template

from anguilla.fitness import benchmark

In [ ]:
# Search space dimension for both constrained and non-rotated benchmark functions is 30.

D_CONSTRAINED_NONROTATED = 30

# Search space dimension for rotated benchmark functions is 10.

D_ROTATED = 10

N_PARENTS = 100

N_TRIALS = 25

# The cartesian product of the following three global variables represents the space of 
# configured MOCMA-Benchmark function pairs for which we need to run N_TRIALS each

# We put the explicit expressions to compute the initial step size 
# for constrained functions as in p. 489 [1].
FUNCTIONS = [
    (benchmark.ZDT1, [D_CONSTRAINED_NONROTATED], None, 0.6 * (1.0 - 0.0)),
    (benchmark.ZDT2, [D_CONSTRAINED_NONROTATED], None, 0.6 * (1.0 - 0.0)),
    (benchmark.ZDT3, [D_CONSTRAINED_NONROTATED], None, 0.6 * (1.0 - 0.0)),
    (benchmark.ZDT4, [D_CONSTRAINED_NONROTATED], None, 0.6 * (1.0 - 0.0)),
    (benchmark.ZDT6, [D_CONSTRAINED_NONROTATED], None, 0.6 * (1.0 - 0.0)),
    (benchmark.IHR1, [D_ROTATED], None, 0.6 * (1.0 - -1.0)),
    (benchmark.IHR2, [D_ROTATED], None, 0.6 * (1.0 - -1.0)),
    (benchmark.IHR3, [D_ROTATED], None, 0.6 * (1.0 - -1.0)),
    (benchmark.IHR4, [D_ROTATED], None, 0.6 * (5.0 - -5.0)),
    (benchmark.IHR6, [D_ROTATED], None, 0.6 * (5.0 - -5.0)),
    (benchmark.ELLI1, [D_ROTATED], {'a':1e6}, 1.0),
    (benchmark.ELLI2, [D_ROTATED], {'a':1e6}, 1.0),
    (benchmark.CIGTAB1, [D_ROTATED], {'a':1e6}, 1.0),
    (benchmark.CIGTAB2, [D_ROTATED], {'a':1e-6}, 1.0),
    (benchmark.DTLZ1, [D_CONSTRAINED_NONROTATED, 3], None, 0.6 * (1.0 - 0.0)),
    (benchmark.DTLZ2, [D_CONSTRAINED_NONROTATED, 3], None, 0.6 * (1.0 - 0.0)),
    (benchmark.DTLZ3, [D_CONSTRAINED_NONROTATED, 3], None, 0.6 * (1.0 - 0.0)),
    (benchmark.DTLZ4, [D_CONSTRAINED_NONROTATED, 3], None, 0.6 * (1.0 - 0.0)),
    (benchmark.DTLZ5, [D_CONSTRAINED_NONROTATED, 3], None, 0.6 * (1.0 - 0.0)),
    (benchmark.DTLZ6, [D_CONSTRAINED_NONROTATED, 3], None, 0.6 * (1.0 - 0.0)),
    (benchmark.DTLZ7, [D_CONSTRAINED_NONROTATED, 3], None, 0.6 * (1.0 - 0.0)),
    (benchmark.GELLI, [D_ROTATED, 3], None, 1.0),
]

OPTS = [
    '(100+1)-MO-CMA-ES-I',
    '(100+1)-MO-CMA-ES-P',
    '(100+100)-MO-CMA-ES-I',
    '(100+100)-MO-CMA-ES-P',
    #'NSGAII',
]

In [ ]:
# Based on: http://eosrei.net/articles/2015/11/latex-templates-python-and-jinja2-generate-pdfs

latex_jinja_env = jinja2.Environment(
    block_start_string = '\BLOCK{',
    block_end_string = '}',
    variable_start_string = '\VAR{',
    variable_end_string = '}',
    comment_start_string = '\#{',
    comment_end_string = '}',
    line_statement_prefix = '%%',
    line_comment_prefix = '%#',
    trim_blocks = True,
    autoescape = False,
    loader = jinja2.FileSystemLoader(pathlib.Path('.'))
)

In [ ]:
template = latex_jinja_env.get_template('fn_page.tex')
output_path = pathlib.Path("./pages")
upmocma_path = pathlib.Path("../experiments/upmocma/evolution")

for cfg in FUNCTIONS:
    fn_cls, fn_args, fn_kwargs, initial_step_size = cfg
    if fn_kwargs is None:
        fn_kwargs = {}
    fn = fn_cls(*fn_args, **fn_kwargs)
    rendered_template = template.render(fn=fn,
                                        opts=OPTS,
                                        fig_ext="pdf" if fn.n_objectives < 3 else "png",
                                        initial_step_size=initial_step_size,
                                        display_upmocma=upmocma_path.joinpath(f"{fn.name}/").exists()
                                       )
    with open(output_path.joinpath(f"{fn.name}.tex"), "w") as outputfile:
        outputfile.write(rendered_template)
    print(f"Wrote {fn.name}")